In [161]:
# mangling
class A:
    __x = 1

a = A()
a._A__x

1

In [72]:
class Ten:
    def __get__(self, obj, objtype=None):
        return 10

In [197]:
class A:
    x = 5
    y = Ten()
    

In [198]:
a = A()
a.x
a.y

10

In [144]:
import os

class DirectorySize:

    def __get__(self, obj, objtype=None):
        print(obj.dirname)
        return len(os.listdir(obj.dirname))

class Directory:

    size = DirectorySize()              # Descriptor instance

    def __init__(self, dirname):
        self.dirname = dirname          # Regular instance attribute

In [150]:
directory = Directory('my_dir ')

In [151]:
directory.size

my_dir 


0

In [156]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAgeAccess:

    def __get__(self, obj, objtype=None):
        value = obj._age
        logging.info('Accessing %r giving %r', 'age', value)
        return value

    def __set__(self, obj, value):
        logging.info('Updating %r to %r', 'age', value)
        obj._age = value

class Person:

    age = LoggedAgeAccess()             # Descriptor instance

    def __init__(self, name, age):
        self.name = name                # Regular instance attribute
        self.age = age                  # Calls __set__()

    def birthday(self):
        self.age += 1                   # Calls both __get__() and __set__()

In [159]:
person = Person('김명지',25)
person.age = 10
person.age

INFO:root:Updating 'age' to 25
INFO:root:Updating 'age' to 10
INFO:root:Accessing 'age' giving 10


10

In [ ]:
# 디스크립터는 __get__(), __set__() 또는 __delete__()를 정의하는 모든 객체를 우리가 부르는 이름

In [192]:
class MyDiscriptor:
    def __get__(self,obj, objtype=None):
        owner = obj.owner
        return f"this is {owner}'s discriptor"
        
    def __set__(self, obj, value):
        obj.owner = value

    def __delete__(self, obj):
        del obj.owner
        obj.owner = 'new_owner'
    def __set_name__(self,owner,name):
        print('this is __set_name__')
        pass

class MyClass:
    owner = 'default_owner'
    my_discriptor = My_Discriptor()
    

In [193]:
myclass = MyClass()

In [ ]:
# 디스크립터는 클래스 변수로 사용될 때만 작동합니다. 인스턴스에 넣으면, 효과가 없습니다.

In [ ]:
# 디스크립터의 주요 동기는 클래스 변수에 저장된 객체가 
# 어트리뷰트 조회 중에 발생하는 일을 제어 할 수 있도록 하는 훅을 제공하는 것입니다.

In [274]:
from abc import ABC, abstractmethod

class Validator(ABC):

    def __set_name__(self, owner, name):
        self.private_name = '_' + name

    def __get__(self, obj, objtype=None):
        # getattr(obj, name) : obj의 어트리뷰트중 이름이 name인 어트리뷰트를 반환
        return getattr(obj, self.private_name) 

    def __set__(self, obj, value):
        self.validate(value)
        setattr(obj, self.private_name, value)

    @abstractmethod
    def validate(self, value):
        pass

In [275]:
import sys
import re

class OneOf(Validator):

    def __init__(self, *options):
        super(Validator).__init__()
        self.options = set(options)

    def validate(self, value):
        if value not in self.options:
            raise ValueError(f'Expected {value!r} to be one of {self.options!r}')

class Number(Validator):

    def __init__(self, minvalue=None, maxvalue=None):
        self.minvalue = minvalue
        self.maxvalue = maxvalue

    def validate(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError(f'Expected {value!r} to be an int or float')
        if self.minvalue is not None and value < self.minvalue:
            raise ValueError(
                f'Expected {value!r} to be at least {self.minvalue!r}'
            )
        if self.maxvalue is not None and value > self.maxvalue:
            raise ValueError(
                f'Expected {value!r} to be no more than {self.maxvalue!r}'
            )

class String(Validator):

    def __init__(self, minsize=None, maxsize=None, predicate=None):
        self.minsize = minsize
        self.maxsize = maxsize
        self.predicate = predicate

    def validate(self, value):
        if not isinstance(value, str):
            raise TypeError(f'Expected {value!r} to be an str')
        if self.minsize is not None and len(value) < self.minsize:
            raise ValueError(
                f'Expected {value!r} to be no smaller than {self.minsize!r}'
            )
        if self.maxsize is not None and len(value) > self.maxsize:
            raise ValueError(
                f'Expected {value!r} to be no bigger than {self.maxsize!r}'
            )
        if self.predicate is not None and not self.predicate(value):
            raise ValueError(
                # r이 의미하는 것은 객체 value의 __reqr__ 호출을 의미한다.
                f'Expected {self.predicate} to be true for {value!r}' 
            )

class HexaCode(Validator):
    def __init__(self, hexacode=b'0x0000', minbyte=4, maxbyte=40):
        super(Validator).__init__()
        self.minbyte = minbyte
        self.maxbyte = maxbyte
        self.hexacode = hexacode

    def validate(self, value):
        hexacode_size = sys.getsizeof(value)
        print(f'{value!r}')
        if not re.match(r'^0[xX][0-9a-fA-F]+$',value_str):
            raise ValueError(
                 f'Expected {value!r} to be Hexacode'
            )
        if hexacode_size > self.maxbyte:
            raise ValueError(
                 f'Expected {value!r} to be no bigger than {self.maxbyte!r}'
            )
        if hexacode_size < self.minbyte:
            raise ValueError(
                 f'Expected {value!r} to be no smaller than {self.minbyte!r}'
            )
        
    

In [276]:
class Component:

    name = String(minsize=3, maxsize=10, predicate=str.isupper)
    kind = OneOf('wood', 'metal', 'plastic')
    quantity = Number(minvalue=0)
    hexacode = HexaCode(hexacode=b'0x000')

    def __init__(self, name, kind, quantity, hexacode):
        self.name = name
        self.kind = kind
        self.quantity = quantity
        self.hexacode = hexacode

In [277]:
c = Component('MYCOMP','wood',4, b'0x124FD')

b'0x124FD'


In [278]:
vars(c)

{'_name': 'MYCOMP', '_kind': 'wood', '_quantity': 4, '_hexacode': b'0x124FD'}

<re.Match object; span=(0, 7), match='0x124FD'>
